# Wie lernt eine KI? Ein praktisches Beispiel

Stell dir vor, du zeigst einem Kind Fotos von drei verschiedenen Blumenarten. Nach einer Weile kann das Kind neue Blumen richtig zuordnen, ohne sie vorher gesehen zu haben. Genau so funktioniert maschinelles Lernen!

Heute werden wir:
1. **Verstehen**, woher unsere Daten kommen
2. **Erkunden**, wie diese Daten aussehen
3. **Trainieren** eine KI, die Blumen unterscheiden kann
4. **Testen**, ob die KI wirklich gelernt hat

---

## 📚 Schritt 1: Die Geschichte unserer Daten

### Der Iris-Datensatz

Unsere Daten stammen aus dem Jahr **1936**! Ein britischer Biologe namens **Ronald Fisher** hat drei Arten von Iris-Blumen untersucht:

**Iris setosa** (Borsten-Schwertlilie)

<img src="data/iris_setosa.png" alt="drawing" width="400"/>

**Iris versicolor** (Verschiedenfarbige Schwertlilie)

<img src="data/iris_versicolor.jpg" alt="drawing" width="400"/>


**Iris virginica** (Virginische Schwertlilie)

<img src="data/iris_virginica_2.jpg" alt="drawing" width="400"/>


Er hat von jeder Art **50 Blumen** genommen und vier Dinge gemessen:

1. **Kelchblatt-Länge** (sepal length): Die langen grünen Blätter außen
2. **Kelchblatt-Breite** (sepal width): Wie breit diese Blätter sind
3. **Blütenblatt-Länge** (petal length): Die bunten Blätter innen
4. **Blütenblatt-Breite** (petal width): Wie breit diese sind

**Insgesamt: 150 Blumen × 4 Messungen = 600 Zahlen**

## 🔧 Schritt 2: Module vorbereiten

Bevor wir starten, brauchen wir zuerset die nötigen Python Module. Das sind fertige Programmteile, die andere Leute geschrieben haben.

**Was macht welches Modul?**
- `numpy`: Rechnen mit vielen Zahlen gleichzeitig
- `pandas`: Daten in Tabellen anzeigen
- `matplotlib`: Diagramme zeichnen
- `sklearn`: Die KI-Werkzeuge!

In [ ]:
import sys

# Installation der Module
if 'pyodide' in sys.modules:
    import piplite
    print("⏳ Installiere Pakete für JupyterLite...")
    await piplite.install(['numpy', 'pandas', 'matplotlib', 'scikit-learn'])
    print("✓ Fertig!")

# Import der Module
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

print("✓ Alle Module erfolgreich importiert!")

## 📊 Schritt 3: Daten laden und verstehen

Jetzt laden wir den Iris-Datensatz. Er ist bereits in scikit-learn eingebaut.

In [ ]:
# Datensatz laden
iris = load_iris()

print("📦 Datensatz geladen!")
print("\n🔍 Was ist drin?")
print(f"   • Anzahl Blumen: {len(iris.data)}")
print(f"   • Anzahl Merkmale pro Blume: {len(iris.data[0])}")
print(f"   • Blumenarten: {iris.target_names}")
print(f"\n📏 Die vier Messungen:")
for i, name in enumerate(iris.feature_names):
    print(f"   {i+1}. {name}")

### Die Daten als Tabelle anschauen

Zahlen in Listen sind schwer zu lesen. Schauen wir sie uns als Tabelle an!

In [ ]:
# Daten in eine schöne Tabelle verwandeln
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['Blumenart'] = pd.Categorical.from_codes(iris.target, iris.target_names)

print("🌸 Die ersten 10 Blumen in unserem Datensatz:\n")
print(df.head(10))

print("\n📈 Statistik zu unseren Daten:")
print(df.describe())

### 💭 Was sehen wir hier?

Jede Zeile ist eine Blume. Die ersten vier Spalten sind die **Messungen** (in Zentimetern), die letzte Spalte ist die **Blumenart**.

**Beispiel Zeile 0:**
- Kelchblatt: 5.1 cm lang, 3.5 cm breit
- Blütenblatt: 1.4 cm lang, 0.2 cm breit
- → Das ist eine **Setosa**

Die KI soll lernen: "Wenn die Messungen so aussehen, dann ist es wahrscheinlich eine Setosa!"

## 📊 Schritt 4: Daten visualisieren

Ein Bild sagt mehr als tausend Zahlen! Schauen wir uns an, ob die Blumenarten wirklich unterschiedlich sind.

In [ ]:
# Streudiagramm: Kelchblatt-Maße
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

colors = ['red', 'green', 'blue']
X = iris.data
y = iris.target

# Diagramm 1: Kelchblatt
for i, species in enumerate(iris.target_names):
    ax1.scatter(X[y == i, 0], X[y == i, 1], 
                c=colors[i], label=species, alpha=0.7, s=80)
ax1.set_xlabel('Kelchblatt-Länge (cm)', fontsize=11)
ax1.set_ylabel('Kelchblatt-Breite (cm)', fontsize=11)
ax1.set_title('Kelchblatt-Maße', fontsize=13, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Diagramm 2: Blütenblatt
for i, species in enumerate(iris.target_names):
    ax2.scatter(X[y == i, 2], X[y == i, 3], 
                c=colors[i], label=species, alpha=0.7, s=80)
ax2.set_xlabel('Blütenblatt-Länge (cm)', fontsize=11)
ax2.set_ylabel('Blütenblatt-Breite (cm)', fontsize=11)
ax2.set_title('Blütenblatt-Maße', fontsize=13, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Was bedeutet das?")
print("   • Die drei Farben sind die drei Blumenarten")
print("   • Jeder Punkt ist eine Blume")
print("   • Die Arten bilden unterschiedliche 'Wolken' → sie sind unterscheidbar!")
print("\n🎯 Das ist das Muster, das die KI lernen soll.")

### 🤔 Warum ist das wichtig?

Schau dir das rechte Diagramm an (Blütenblatt). Die roten Punkte (Setosa) sind **weit weg** von den anderen. Das bedeutet: Setosa ist leicht zu erkennen!

Die grünen und blauen Punkte überlappen etwas. Das wird schwieriger für die KI.

## ✂️ Schritt 5: Daten aufteilen

### Das Schul-Prinzip

Stell dir vor, du lernst für eine Prüfung:
- **Übungsaufgaben** → damit lernst du (= Trainingsdaten)
- **Prüfungsaufgaben** → damit wird getestet, ob du's verstanden hast (= Testdaten)

Wenn die Prüfung die gleichen Aufgaben hat wie beim Üben, wüssten wir nicht, ob du wirklich verstanden hast oder nur auswendig gelernt hast!

**Deshalb teilen wir unsere 150 Blumen auf:**
- 80% (120 Blumen) → Training
- 20% (30 Blumen) → Test

In [ ]:
# Daten aufteilen
X = iris.data  # Die Messungen
y = iris.target  # Die richtigen Antworten (0, 1, oder 2)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("📚 Daten aufgeteilt!")
print(f"\n   Trainingsdaten: {len(X_train)} Blumen (zum Lernen)")
print(f"   Testdaten: {len(X_test)} Blumen (zum Überprüfen)")
print(f"\n   Verhältnis: {len(X_train)/len(X)*100:.0f}% Training, {len(X_test)/len(X)*100:.0f}% Test")

## 📏 Schritt 6: Normalisierung - Alle auf eine Skala bringen

### Das Problem

Stell dir vor, du vergleichst:
- Die Geschwindigkeit eines Autos: 120 km/h
- Das Gewicht eines Autos: 1500 kg

Die Zahlen sind sehr unterschiedlich groß! Die KI könnte denken, dass das Gewicht wichtiger ist, nur weil die Zahl größer ist.

### Die Lösung: Normalisierung

Wir verwandeln alle Werte so, dass sie ungefähr zwischen -2 und +2 liegen. Dann sind alle Merkmale gleich wichtig.

In [ ]:
# Normalisierung
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("📏 Normalisierung durchgeführt!\n")
print("Vorher (erste Trainingsblume):")
print(f"   {X_train[0]}")
print(f"   → Werte zwischen {X_train[0].min():.1f} und {X_train[0].max():.1f}")
print("\nNachher:")
print(f"   {X_train_scaled[0]}")
print(f"   → Werte zwischen {X_train_scaled[0].min():.1f} und {X_train_scaled[0].max():.1f}")
print("\n✅ Jetzt sind alle Werte auf ähnlicher Skala!")

## 🧠 Schritt 7: Das neuronale Netz erstellen

### Was ist ein neuronales Netz?

Ein neuronales Netz ist wie ein Team von Entscheidungshelfern:

```
Eingabe (4 Zahlen)    Versteckte Schicht    Ausgabe (3 Zahlen)
                      
Kelchblatt-Länge ──┐  
Kelchblatt-Breite ─┼──→ [10 Helfer] ──┐
Blütenblatt-Länge ─┼──→ [10 Helfer] ──┼──→ Setosa?
Blütenblatt-Breite ┘                   ├──→ Versicolor?
                                       └──→ Virginica?
```

- **Eingabe**: Die 4 Messungen
- **Versteckte Schichten**: Hier passiert das "Denken"
- **Ausgabe**: Wahrscheinlichkeit für jede Blumenart

In [ ]:
# Neuronales Netz erstellen
model = MLPClassifier(
    hidden_layer_sizes=(10, 10),  # Zwei Schichten mit je 10 "Helfern"
    max_iter=1000,                # Maximal 1000 Lernrunden
    random_state=42,              # Für reproduzierbare Ergebnisse
    verbose=False                 # Nicht zu viel Text ausgeben
)

print("🧠 Neuronales Netz erstellt!")
print("\n📐 Architektur:")
print("\n   [4 Eingänge] → [10 Neuronen] → [10 Neuronen] → [3 Ausgänge]")
print("\n   • 4 Eingänge: Die 4 Messungen")
print("   • 2× 10 Neuronen: Hier lernt das Netz Muster")
print("   • 3 Ausgänge: Eine für jede Blumenart")
print("\n🎲 Das Netz ist jetzt wie ein Baby - es kann noch nichts!")

## 🚀 Schritt 8: Training - Jetzt lernt die KI!

### Was passiert beim Training?

1. Die KI schaut sich eine Blume an
2. Sie rät, welche Art es ist
3. Wir sagen ihr, ob sie richtig lag
4. Sie passt ihre inneren "Verbindungen" an
5. Beim nächsten Mal rät sie hoffentlich besser

Das macht sie 1000 Mal mit allen 120 Trainingsblumen!

In [ ]:
print("⏳ Training startet... (kann 5-10 Sekunden dauern)\n")

# Training durchführen
model.fit(X_train_scaled, y_train)

print("\n✅ Training abgeschlossen!")
print("\n🎓 Die KI hat jetzt aus 120 Beispielen gelernt.")
print("   Mal schauen, ob sie es verstanden hat...")

## 🎯 Schritt 9: Die große Prüfung!

Jetzt zeigen wir der KI die 30 Testblumen, die sie noch NIE gesehen hat.

In [ ]:
# Vorhersagen machen
y_pred = model.predict(X_test_scaled)

# Genauigkeit berechnen
accuracy = accuracy_score(y_test, y_pred)
richtig = (y_test == y_pred).sum()
gesamt = len(y_test)

print("🎯 ERGEBNIS DER PRÜFUNG\n")
print(f"   Richtig erkannt: {richtig} von {gesamt} Blumen")
print(f"   Genauigkeit: {accuracy * 100:.1f}%")
print(f"\n   Note: ", end="")
if accuracy >= 0.95:
    print("⭐⭐⭐ AUSGEZEICHNET! Die KI hat es wirklich verstanden!")
elif accuracy >= 0.85:
    print("⭐⭐ SEHR GUT! Ein paar kleine Fehler.")
elif accuracy >= 0.75:
    print("⭐ GUT. Die KI braucht vielleicht mehr Training.")
else:
    print("Hmm, da ist was schiefgegangen...")

### Einzelne Vorhersagen anschauen

In [ ]:
print("\n🔍 Schauen wir uns ein paar Vorhersagen im Detail an:\n")
print("═" * 60)

for i in range(min(15, len(y_test))):
    actual = iris.target_names[y_test[i]]
    predicted = iris.target_names[y_pred[i]]
    
    if y_test[i] == y_pred[i]:
        status = "✅"
        comment = "Richtig!"
    else:
        status = "❌"
        comment = f"Fehler! Dachte es wäre {predicted}"
    
    print(f"{status} Blume #{i+1}: {actual:12} → {comment}")

## 📊 Schritt 10: Wo macht die KI Fehler?

Die **Confusion Matrix** (Verwirrungsmatrix) zeigt uns, welche Arten die KI verwechselt.

In [ ]:
# Confusion Matrix erstellen
cm = confusion_matrix(y_test, y_pred)

# Visualisieren
fig, ax = plt.subplots(figsize=(9, 7))
im = ax.imshow(cm, cmap='Blues')

# Achsen beschriften
ax.set_xticks(np.arange(len(iris.target_names)))
ax.set_yticks(np.arange(len(iris.target_names)))
ax.set_xticklabels(iris.target_names, fontsize=11)
ax.set_yticklabels(iris.target_names, fontsize=11)

# Zahlen in die Felder schreiben
for i in range(len(iris.target_names)):
    for j in range(len(iris.target_names)):
        text_color = "white" if cm[i, j] > cm.max()/2 else "black"
        ax.text(j, i, cm[i, j], ha="center", va="center", 
                color=text_color, fontsize=18, fontweight='bold')

ax.set_ylabel('Tatsächliche Blumenart', fontsize=12)
ax.set_xlabel('Von der KI vorhergesagt', fontsize=12)
ax.set_title('Confusion Matrix: Welche Fehler macht die KI?', fontsize=14, fontweight='bold', pad=20)
plt.colorbar(im, ax=ax)
plt.tight_layout()
plt.show()

print("\n💡 So liest du die Matrix:")
print("\n   • Zahlen auf der DIAGONALE (↘) = Richtige Vorhersagen")
print("   • Zahlen NEBEN der Diagonale = Fehler")
print("\n📊 Beispiel:")
print("   Die Zahl oben links zeigt: Wie viele Setosa wurden richtig erkannt?")
print("   Zahlen daneben zeigen: Wurde eine Setosa mit was anderem verwechselt?")

## 🌺 Schritt 11: Teste die KI mit einer neuen Blume!

Jetzt kommt der spannende Teil: Wir "erfinden" eine Blume und schauen, was die KI sagt.

In [ ]:
# Eine neue Blume zum Testen
# Format: [Kelchblatt-Länge, Kelchblatt-Breite, Blütenblatt-Länge, Blütenblatt-Breite]
neue_blume = np.array([[5.1, 3.5, 1.4, 0.2]])

# Normalisieren (gleiche Transformation wie beim Training)
neue_blume_scaled = scaler.transform(neue_blume)

# Vorhersage machen
vorhersage = model.predict(neue_blume_scaled)
wahrscheinlichkeiten = model.predict_proba(neue_blume_scaled)

print("🌺 TEST: Eine neue Blume klassifizieren\n")
print("═" * 50)
print("\n📏 Messungen der Blume:")
print(f"   • Kelchblatt: {neue_blume[0][0]} cm lang, {neue_blume[0][1]} cm breit")
print(f"   • Blütenblatt: {neue_blume[0][2]} cm lang, {neue_blume[0][3]} cm breit")

print(f"\n🤖 Die KI sagt: Das ist eine '{iris.target_names[vorhersage[0]]}'")

print("\n📊 Wie sicher ist sich die KI?")
for i, species in enumerate(iris.target_names):
    prozent = wahrscheinlichkeiten[0][i] * 100
    balken = "█" * int(prozent / 5)
    print(f"   {species:12}: {balken} {prozent:.1f}%")

print("\n💭 Interpretiere das Ergebnis:")
max_prob = wahrscheinlichkeiten[0].max() * 100
if max_prob > 90:
    print("   Die KI ist sich sehr sicher!")
elif max_prob > 70:
    print("   Die KI ist ziemlich sicher, aber nicht 100%.")
else:
    print("   Die KI ist unsicher - die Blume könnte mehrere Arten sein.")

### 🎮 Experimentiere selbst!

Ändere die Werte in der Zeile `neue_blume = np.array([[...]])` und führe die Zelle nochmal aus!

**Beispiele zum Ausprobieren:**
```python
# Typische Setosa (kleine Blütenblätter):
neue_blume = np.array([[5.0, 3.5, 1.3, 0.3]])

# Typische Versicolor (mittlere Größe):
neue_blume = np.array([[6.0, 2.7, 4.5, 1.5]])

# Typische Virginica (große Blütenblätter):
neue_blume = np.array([[6.5, 3.0, 5.5, 2.0]])

# Extreme Werte (was passiert?):
neue_blume = np.array([[8.0, 4.0, 7.0, 3.0]])
```

## 🎓 Zusammenfassung: Was haben wir gelernt?

### Der komplette Ablauf:

1. **📚 Daten verstehen**
   - Woher kommen sie? (Iris-Datensatz von 1936)
   - Was bedeuten die Zahlen? (Blumen-Messungen)
   - Sind die Klassen unterscheidbar? (Ja, siehe Diagramme!)

2. **✂️ Daten vorbereiten**
   - Training/Test aufteilen (80/20)
   - Normalisieren (alle Werte auf ähnliche Skala)

3. **🧠 Modell erstellen**
   - Neuronales Netz mit 2 versteckten Schichten
   - Anfangs weiß es noch nichts!

4. **🚀 Training**
   - KI lernt aus 120 Beispielen
   - Passt ihre internen Verbindungen an

5. **🎯 Test**
   - Prüfung mit 30 neuen Blumen
   - Meist 95-100% Genauigkeit!

6. **🌺 Anwendung**
   - Neue Blumen klassifizieren
   - Wahrscheinlichkeiten verstehen

---

### 💡 Wichtige Konzepte:

| Konzept | Was bedeutet das? |
|---------|-------------------|
| **Training** | Die KI lernt aus Beispielen |
| **Test** | Überprüfung mit neuen, ungesehenen Daten |
| **Features** | Die Merkmale (hier: 4 Messungen) |
| **Labels** | Die richtigen Antworten (hier: Blumenarten) |
| **Overfitting** | Wenn die KI auswendig lernt statt Muster zu erkennen |
| **Accuracy** | Wie oft liegt die KI richtig? |

---

### 🚀 Nächste Schritte:

**Experimentiere weiter:**
- Ändere die Netzwerk-Größe: `hidden_layer_sizes=(5, 5)` oder `(20, 20, 20)`
- Ändere die Lernrunden: `max_iter=500` oder `max_iter=2000`
- Was passiert mit nur 50% Trainingsdaten? Ändere: `test_size=0.5`

**Vertiefe dein Wissen:**
- Wie funktionieren neuronale Netze genau?
- Was sind andere Machine-Learning-Algorithmen? (Decision Trees, Random Forest, SVM)
- Wie trainiert man KI mit Bildern oder Text?

---

### 🎉 Glückwunsch!

Du hast gerade:
- ✅ Einen echten Datensatz verstanden
- ✅ Daten für KI vorbereitet
- ✅ Ein neuronales Netz trainiert
- ✅ Die Qualität getestet
- ✅ Das Modell angewendet

**Das ist genau der Prozess, den auch professionelle Data Scientists verwenden!**

---

### 📚 Ressourcen zum Weiterlernen:

- **Scikit-learn Dokumentation**: https://scikit-learn.org/
- **Kaggle Learn**: Kostenlose Kurse zu Machine Learning
- **Fast.ai**: Praktische Deep Learning Kurse
- **3Blue1Brown**: Visuelle Erklärungen zu neuronalen Netzen (YouTube)

---

**💪 Viel Erfolg auf deiner KI-Reise!**